### 构造

In [1]:
!pip install unstructured
!pip install python-doc
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 976.2 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=999638e4e75e0b1683b10732b36be3b1a75e9a63f3806c5ffedca9fd3283f790
  Stored in directory: /Users/sanford/Library/C

In [1]:
import torch
import chatglm_cpp
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import  Language
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if torch.cuda.is_available():
    !pip install ctransformers[cuda]
    
DEVICE = 'cpu'
# DEVICE

'mps'

文档解析

In [2]:
loader = DirectoryLoader(
    'data/pdf',
    glob="*.PDF",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
    )

documents = []
for file in loader.lazy_load():
    try:
        documents.append(file)
    except:
        print(file)

# loader = DirectoryLoader(
#     'data/pdf',
#     glob="*.PDF",
#     loader_cls=PyPDFLoader,
#     show_progress=True,
#     use_multithreading=True
#     )
# documents = loader.load()

documents[0]

100%|██████████| 1/1 [00:06<00:00,  6.66s/it]


Document(page_content=' \n \n  \n \n \n \n \n \n \n \n \n \n \n平安银行股份有限公司  \n \n2021年年度报告  \n \n  \n', metadata={'source': 'data/pdf/000001∆Ω∞≤“¯––_2021ƒÍƒÍ∂»±®∏Ê_2022-03-10_SZZB.PDF', 'page': 0})

文档分割

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

split_docs[:3]

[Document(page_content='平安银行股份有限公司  \n \n2021年年度报告', metadata={'source': 'data/pdf/000001∆Ω∞≤“¯––_2021ƒÍƒÍ∂»±®∏Ê_2022-03-10_SZZB.PDF', 'page': 0}),
 Document(page_content='重要提示  \n1 \n 平安银行股份有限公司  \n2021年年度报告 \n重要提示  \n1、本行董事会、监事会及董事、监事、高级管理人员保证年度报告内容的真实、准确、完整，\n不存在虚假记载、误导性陈述或重大遗漏，并承担个别和连带的法律责任。  \n2、本行董事长谢永林、 行长胡跃飞、 副行长兼首席财务官项有志、 会计机构负责人朱培卿声明：\n保证本年度报告中财务报告的真实、准确、完整。  \n3、本行第十一届董事会第 二十九次会议审议了 2021年年度报告正文及摘要。 本次董事会会议\n应出席董事 14人，实到董事14人。会议一致同意本报告。  \n4、安永华明 会计师事务所（特殊普通合伙）根据国内审计准则对 本行编制的 2021年度财务报\n告进行了审计，并出具了标准无保留意见的审计报告。  \n5、本报告涉及未来计划等前瞻性陈述不构成本行对投资者的实质承诺，投资者及相关人士均应\n当对此保持足够的风险认识，并且应当理解计划、预测与承诺之间的差异。  \n6、本行请投资者认真阅读本年度报告全文，并特别注意下列风险因素：本 行经营中面临的风险\n主要有信用风险、市场风险、流动性风险、操作风险、国别风险、银行账簿 利率风险、声誉风险、\n战略风险、信息科技风险以及法律和合规风险等，本 行已经采取各种措施，有效管理和控制各类经\n营风险，具体详见 “第三章  3.6 风险管理 ”。 \n7、本行经本次董事会审议通过的普通股利润分配预案为：以本行 2021年12月31日的总股本\n19,405,918,198 股为基数，向全体股东每 10股派发现金 股利人民币 2.28元（含税） ，不送红股，不以\n公积金转增股本。', metadata={'source': 'data/pdf/000001∆Ω∞≤“¯––_2021ƒÍƒÍ∂»±®∏Ê_2022-03-10_SZZB.PDF', 'page': 1}),
 Document

向量嵌入

In [4]:
embedding_path = "model/text2vec-base-chinese"
model_kwargs = {'device': DEVICE}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder=embedding_path
)

query_result = embeddings.embed_query("平安银行股份有限公司")
query_result[:3]

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name model/text2vec-base-chinese. Creating a new one with mean pooling.


[0.06715575605630875, 0.012016350403428078, -0.023342687636613846]

向量保存

In [ ]:
vs_path = 'model/vectors/faiss'
db = FAISS.from_documents(split_docs, embeddings)
db.save_local(vs_path)


检索

In [6]:
vs_path = 'model/vectors/faiss'
vector_store = FAISS.load_local(
    vs_path,
    embeddings,
    allow_dangerous_deserialization=True
    )

related_docs_with_score = vector_store.similarity_search_with_score(query="重要提示", k=3)
context = ""
for pack in related_docs_with_score:
    doc, socre = pack
    content = doc.page_content
    print("检索到的知识=%s, from=%s, socre=%.3f"%(content[:50], doc.metadata.get("from"), socre))
    context += content

检索到的知识=2021年年度报告  
3 / 325 
 十、 重大风险提示  
公司在“管理层讨论与分析 ”一节, from=None, socre=0.946
检索到的知识=2021年年度报告  
3 / 261 
 十、 重大风险提示  
公司已在本 报告中详细描述存在的, from=None, socre=1.005
检索到的知识=2021年年度报告  
3 / 193 
 十、 重大风险提示  
报告期内，公司不存在对生产经营产, from=None, socre=1.019


### 使用

ChatGLM3

In [ ]:
# chatglm_path = "model/chatglm/chatglm3-6b-q4_0-ggml.bin"
# pipeline = chatglm_cpp.Pipeline(chatglm_path)
# pipeline.chat([chatglm_cpp.ChatMessage(role="user", content="你好")])

In [ ]:
# %%script bash
# CMAKE_ARGS="-DGGML_CUBLAS=ON" pip install -U chatglm-cpp

后台chatGLM3服务

In [3]:
%%script bash --bg
MODEL=model/chatglm/chatglm3-6b-q4_0-ggml.bin uvicorn chatglm_cpp.langchain_api:app --host 127.0.0.1 --port 8000 &

In [7]:
from langchain.llms import ChatGLM
llm = ChatGLM(endpoint_url="http://127.0.0.1:8000")
llm.predict("你好")

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'你好👋！我是人工智能助手 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。'

提示词

In [4]:
template ="""根据下面上下文（context）内容回答问题。
如果你不知道答案，就回答不知道，不要试图编造答案。
答案最多3句话，保持答案简洁。
总是在答案结束时说“谢谢你的提问！”
{context}
问题：{question}
答案："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

检索问答

In [9]:
from langchain.llms import ChatGLM

embedding_path = "model/text2vec-base-chinese"
model_kwargs = {'device': DEVICE}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder=embedding_path
)

vs_path = 'model/vectors/faiss'
vectordb = FAISS.load_local(
    vs_path,
    embeddings,
    allow_dangerous_deserialization=True
    )

qa_chat = RetrievalQA.from_chain_type(
    llm=ChatGLM(endpoint_url="http://127.0.0.1:8000"),
    chain_type='stuff',
    retriever=vectordb.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs={'prompt': QA_CHAIN_PROMPT}
    )


No sentence-transformers model found with name model/text2vec-base-chinese. Creating a new one with mean pooling.


In [10]:
qa_chat("惠州攸特指什么?")

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '惠州攸特指什么?',
 'result': '惠州攸特企业管理中心（有限合伙）是一家位于中国惠州市的企业管理公司，其主要业务可能与企业管理、投资管理等相关。',
 'source_documents': [Document(page_content='湖南凯美特气体股份有限公司 2021年年度报告全文  \n5 释义  \n释义项  指 释义内容  \n本公司、公司  指 湖南凯美特气体股份有限公司  \n凯美特有限  指 湖南凯美特干冰有限公司、湖南凯美特气体有限公司  \n惠州凯美特  指 惠州凯美特气体有限公司  \n安庆凯美特  指 安庆凯美特气体有限公司  \n长岭凯美特  指 岳阳长岭凯美特气体有限公司  \n湖南凯美特特气分公司  指 湖南凯美特气体股份有限公司特气分公司  \n安庆凯美特特气分公司  指 安庆凯美特气体有限公司特气分公司  \n海南凯美特  指 海南凯美特气体有限公司  \n福建凯美特  指 福建凯美特气体有限公司  \n凯美特电子特种气体公司  指 岳阳凯美特电子特种稀有气体有限公司  \n浩讯科技、控股股东  指 浩讯科技有限公司  \n新疆信安  指 新疆信安创业投资合伙企业（有限合伙）  \n财信常勤壹号  指 湖南省财信常勤壹号基金合伙企业 (有限合伙 ) \n财信资产  指 湖南省财信资产管理有限公司', metadata={'source': 'data/2021上市企业企业年报/002549ø≠√¿Ãÿ∆¯_2021ƒÍƒÍ∂»±®∏Ê_2022-03-18_SZZB.PDF', 'page': 4}),
  Document(page_content='2021年年度报告  \n5 / 297 \n  \n第一节 释义  \n一、 释义 \n在本报告书中，除非文义另有所指，下列词语具有如下含义：  \n常用词语释义  \n惠泰医疗、本公\n司、公司  指 深圳惠泰医疗器械股份有限公司  \n湖南埃普特  指 湖南埃普特医疗器械有限公司，惠泰医疗全资子公司  \n上海宏桐 、宏桐\n实业 指 上海宏桐实业有限公司，惠泰医疗控股子公司  \n惠泰观澜分公司  指 深圳惠泰医疗器械股份有限公司观澜分公司  \n启明维创  指 启明维创（上海）股权投资中心（有限合伙）  \nQM33  指 QM33 L

In [12]:
qa_chat("泓禧科技公司的联系方式")

{'query': '泓禧科技公司的联系方式',
 'result': '抱歉，关于泓禧科技公司的联系方式，您在提供的信息中并未找到相关内容。请您提供更多关于泓禧科技公司的信息，以便为您提供更准确的联系方式。',
 'source_documents': [Document(page_content='2021年年度报告  \n4 / 201 \n 第一节 释义  \n一、 释义 \n在本报告书中，除非文义另有所指，下列词语具有如下含义：  \n常用词语释义  \n高科磁业  指 宁波韵升高科磁业有限公司  \n韵升强磁  指 宁波韵升强磁材料有限公司  \n包头强磁  指 包头韵升强磁材料有限公司  \n磁性材料  指 宁波韵升磁性材料有限公司  \n磁体元件  指 宁波韵升磁体元件技术有限公司  \n粘结磁体  指 宁波韵升粘结磁体有限公司  \n特种金属  指 宁波韵升特种金属材料有限公司  \n盛磁科技  指 北京盛磁科技有限公司  \n宁波健信  指 宁波健信核磁技术有限公司  \n会计师事务所  指 天衡会计师事务所（特殊普通合伙）  \n上交所 指 上海证券交易所  \n中国证监会  指 中国证券监督管理委员会  \n \n \n第二节 公司简介和主要财务指标  \n一、 公司信息  \n公司的中文名称  宁波韵升股份有限公司  \n公司的中文简称  宁波韵升  \n公司的外文名称  NINGBO YUNSHENG CO.,LTD.  \n公司的法定代表人  竺晓东 \n \n二、 联系人和联系方式  \n 董事会秘书  证券事务代表  \n姓名 项超麟 赵佳凯 \n联系地址  浙江省宁波市鄞州区扬帆路 1号 浙江省宁波市鄞州区扬帆路 1号 \n电话 0574-87776939  0574-87776939  \n传真 0574-87776466  0574-87776466  \n电子信箱  xiangcl@ysweb.com  zhaojk@ysweb.com  \n \n三、 基本情况 简介 \n公司注册地址  浙江省宁波市鄞州区民安路 348号 \n公司办公地址  浙江省宁波市鄞州区扬帆路 1号 \n公司办公地址的邮政编码  315040 \n公司网址  www.yunsheng.com  \n电子信箱  stock@ysweb.com  \n